In [8]:
#set the wd to file location
setwd(getSrcDirectory(function(){})[1])

# Exploratory Data Analysis 

This notebook is to be run 3rd.\
The purpose of this file is to perform initial exploration on the split of training data prior to vetting to suggest the next steps - wrangling, transformations, predictive methods;

In [11]:
#Import Necessary libraries
suppressWarnings({
library("dplyr")
library("tidyverse")})

In [12]:
##Read Data
data<- read.csv("./03a-vehicles-sample.csv", header = TRUE)

check the dimensions of the data frame

In [13]:
dim(data)

[1] 195064     14

In [14]:
colnames(data)

[1] "price"        "year"         "manufacturer" "condition"    "cylinders"   
 [6] "fuel"         "odometer"     "title_status" "transmission" "drive"       
[11] "size"         "type"         "paint_color"  "state"

### Convert all character data to factors

In [15]:
data<- data%>%
mutate_if(sapply(data, is.character), as.factor)

### Summary Statistics

In [16]:
summary(data)

     price             year         manufacturer       condition    
 Min.   :  1003   Min.   :2000   ford     :31090            :69731  
 1st Qu.:  7500   1st Qu.:2008   chevrolet:23357   excellent:50158  
 Median : 14995   Median :2013   toyota   :16500   fair     : 2776  
 Mean   : 18614   Mean   :2012   honda    :11184   good     :59523  
 3rd Qu.: 26590   3rd Qu.:2017   nissan   : 9526   like new :12023  
 Max.   :199999   Max.   :2022   jeep     : 8311   new      :  612  
                                 (Other)  :95096   salvage  :  241  
        cylinders           fuel           odometer          title_status   
             :75248           :  1041   Min.   :     0             :  2684  
 6 cylinders :46043   diesel  : 12150   1st Qu.: 41526   clean     :184490  
 4 cylinders :40315   electric:   849   Median : 92050   lien      :  1028  
 8 cylinders :30976   gas     :165360   Mean   : 96207   missing   :    94  
 5 cylinders :  968   hybrid  :  2753   3rd Qu.:139056   parts 

### Comments on numeric data:
 - ```year``` : values range from 2000 to 2022 depicting the year the posted vehicle was manufactured. A better variable for interpretation would be ```age```. The dataset was collected for posts made in year 2022 (over the period of 3 months). We can mutate the dataset to create ```age``` as follows $age = 2022 - year$.
  - ```odometer``` : values range from 0 to ~500,000 miles. Extreme values were excluded from the dataset (see 01-dataClean.ipynb), however looking at the summary odometer appears to be skewed to the right. Considering all values are positive, performing a square-root or cube-root transformation might be beneficial. 

### Comments on categorical data 
 - Binning : many attributes have similar categories that can be binned to produce better predictive power. attribute - ```condition```, ```fule```, ```cylinders```, ```transmission```, ```drive```, ```size```, ```type```, ```paint_color``` & ```title_status``` have significant amount of missing values, these may have their own predictive power. we expect a lower asking price if the poster has lesser information about the car. 
 